In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime

In [2]:
url = "https://datos.madrid.es/egob/catalogo/300107-0-agenda-actividades-eventos.json"
response = requests.get(url)
data = response.json()

In [3]:
lista_eventos_api = data.get("@graph")
print(len(lista_eventos_api))


1228


In [4]:
dictio_evento = {"title": [],
                 "url": [],
                 "postal-code": [],
                 "dirección": [],
                 "time": [],
                 "organization": [],
                 "dtstart": [],
                 "dtend": []}
                 

In [5]:

for i in lista_eventos_api:
    nombre_evento = i.get("title")
    dir_url = i.get("link")
    cod_postal = i.get("address", {}).get("area", {}).get("postal-code", "0")
    direccion = i.get("address", {}).get("area", {}).get("street-address", "No disponible")
    horario = i.get ("time", "No disponible")
    organizacion = i.get ("organization", {}).get("organization-name", "No disponible")
    dtstart = i.get("dtstart")
    dtend = i.get ("dtend")

    dictio_evento["title"].append(nombre_evento)
    dictio_evento["url"].append(dir_url)
    dictio_evento["postal-code"].append(cod_postal)
    dictio_evento["dirección"].append(direccion)
    dictio_evento["time"].append(horario)
    dictio_evento["organization"].append(organizacion)
    dictio_evento["dtstart"].append(dtstart)
    dictio_evento["dtend"].append(dtend)
    
df_eventos = pd.DataFrame(dictio_evento)

In [6]:
df_eventos["dtstart"] = pd.to_datetime (df_eventos["dtstart"]).dt.date
df_eventos["dtend"] = pd.to_datetime (df_eventos["dtend"]).dt.date


In [7]:
fecha_inicio = datetime(2025, 3, 2, 0, 0, 0)
fecha_fin = datetime(2025, 3, 3, 23, 59, 59)    

In [8]:
eventos_fechas = []
for _, row in df_eventos.iterrows():
    dtstart_formato = pd.to_datetime(row["dtstart"])
    if fecha_inicio <= dtstart_formato <= fecha_fin:
        eventos_fechas.append(row[["title", "url", "postal-code", "dirección", "time", "organization", "dtstart","dtend"]])

df_eventos_fechas = pd.DataFrame(eventos_fechas)


In [9]:
df_eventos_fechas ["id_ciudad"] = 1


In [10]:
df_eventos_fechas ["postal-code"] = pd.to_numeric (df_eventos_fechas["postal-code"], errors="coerce").astype("Int64")

df_eventos_fechas[["dtstart", "dtend"]] = df_eventos_fechas[["dtstart", "dtend"]].apply(pd.to_datetime)

In [11]:
df_eventos_fechas ["postal-code"].isna().sum()

np.int64(0)

In [12]:
df_eventos_fechas.to_parquet("../data/api_eventos.parquet", index=False)